In [1]:
import numpy as np
import time
from numpy.core.numeric import indices
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle
import pandas as pd
from sklearn import preprocessing 
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold,LeaveOneOut,StratifiedKFold
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import ParameterGrid

In [2]:
def ClassifySpecies(filenames):
    adsorbate_list = []
    for filename in filenames:
        adsorbate  = filename.split('_')[0]
        if adsorbate not in adsorbate_list:
            adsorbate_list.append(adsorbate)
    classify_list = []
    for filename in filenames:
        key = filename.split('_')[0]
        classify_list.append(np.argwhere(np.array(adsorbate_list) == key)[0][0])
    return classify_list
def _data_prepressing(pre_feature_matrix, train_index, test_index, data_processor):
    X_train_raw, X_test_raw = pre_feature_matrix.iloc[train_index], pre_feature_matrix.iloc[test_index]
    hp_processor            = data_processor.fit(X_train_raw)
    X_train, X_test         = hp_processor.transform(X_train_raw), hp_processor.transform(X_test_raw)
    return X_train, X_test
data_preprocessor        = preprocessing.StandardScaler()
def helper_cv(
        params,
        dtrain,
        num_boost_round,
        seed   = 0,
        nfold  = 5,
        metrics= 'rmse',
        early_stopping_rounds = 50,
        **kwargs):
    if kwargs:
        min_rmse = float("Inf")
        best_params = None
        grid = ParameterGrid(kwargs)
        for para_i in grid:
            params.update(para_i)
            cv_results = xgb.cv(
                        params,
                        dtrain,
                        num_boost_round=num_boost_round,
                        seed =seed,
                        nfold=nfold,
                        metrics=metrics,
                        early_stopping_rounds=early_stopping_rounds
                    )   
            mean_rmse = cv_results['test-rmse-mean'].min()
            boost_rounds = cv_results['test-rmse-mean'].argmin()
            print("params:", para_i, "cv score", np.round(mean_rmse,4), "round", boost_rounds)
            if mean_rmse < min_rmse:
                min_rmse = mean_rmse
                best_params = para_i
        print("Best params:", best_params ,min_rmse)
        params.update(best_params)
        return params
    #if not grid search, expect num_boost_round to be optimized
    else:
        cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        nfold  = nfold, 
        metrics= metrics,
        early_stopping_rounds=early_stopping_rounds
        )
        return cv_results.shape[0]

# only perform 1 out of 5-fold as an example

In [3]:
datas                    = pd.read_excel('data.xlsx')
datas_X                  = datas.drop(["Samples", "E_ads"], axis=1)
datas_y                  = datas["E_ads"]
file_names               = datas['Samples']

In [4]:
skf=StratifiedKFold(5, random_state=98, shuffle=True)
for train_index, test_index in skf.split(datas_X, ClassifySpecies(file_names)):
    X_train, X_test = _data_prepressing(datas_X, train_index, test_index, data_preprocessor)
    y_train, y_test = datas_y.iloc[train_index], datas_y.iloc[test_index]

In [5]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest  = xgb.DMatrix(X_test, label=y_test)

# initialize hyperparameters

In [6]:
params = {
         "max_depth"    : 4,
         "min_child_weight": 1,
         "eta" : 0.1, 
         "gamma": 0,
         "reg_lambda" : 1,
         "subsample": 1,
         "colsample_bytree": 1,
          }
num_boost_round = 3000
early_stopping_rounds = 50

# optimize num_boost_round for given learning rate

In [7]:
num_boost_round = helper_cv(
                        params,
                        dtrain,
                        num_boost_round,
                        seed   = 0,
                        nfold  = 5,
                        metrics= 'rmse',
                        early_stopping_rounds = 50)

# grid search for max_depth and min_child_weight

In [8]:
max_depth        = [max_depth        for max_depth in range(3,8,1)]
min_child_weight = [min_child_weight for min_child_weight in range(1,6,1)]
params           = helper_cv(
                        params,
                        dtrain,
                        num_boost_round,
                        seed   = 0,
                        nfold  = 5,
                        metrics= 'rmse',
                        early_stopping_rounds = 50,
                        max_depth = max_depth,
                        min_child_weight = min_child_weight)

params: {'max_depth': 3, 'min_child_weight': 1} cv score 0.1538 round 1337
params: {'max_depth': 3, 'min_child_weight': 2} cv score 0.1528 round 1995
params: {'max_depth': 3, 'min_child_weight': 3} cv score 0.1524 round 1843
params: {'max_depth': 3, 'min_child_weight': 4} cv score 0.1578 round 1440
params: {'max_depth': 3, 'min_child_weight': 5} cv score 0.1575 round 1489
params: {'max_depth': 4, 'min_child_weight': 1} cv score 0.1525 round 1995
params: {'max_depth': 4, 'min_child_weight': 2} cv score 0.1523 round 1995
params: {'max_depth': 4, 'min_child_weight': 3} cv score 0.1498 round 992
params: {'max_depth': 4, 'min_child_weight': 4} cv score 0.1543 round 1037
params: {'max_depth': 4, 'min_child_weight': 5} cv score 0.1559 round 1054
params: {'max_depth': 5, 'min_child_weight': 1} cv score 0.1591 round 1078
params: {'max_depth': 5, 'min_child_weight': 2} cv score 0.1558 round 1203
params: {'max_depth': 5, 'min_child_weight': 3} cv score 0.1568 round 990
params: {'max_depth': 5, 'm

# grid search for subsample and colsample_bytree

In [9]:
subsample        = [subsample/10     for subsample in range(7,11)]
colsample_bytree = [colsample_bytree/10 for colsample_bytree in range(7,11)]
params           = helper_cv(
                        params,
                        dtrain,
                        num_boost_round,
                        seed   = 0,
                        nfold  = 5,
                        metrics= 'rmse',
                        early_stopping_rounds = 50,
                        subsample = subsample,
                        colsample_bytree = colsample_bytree)

params: {'colsample_bytree': 0.7, 'subsample': 0.7} cv score 0.1478 round 1130
params: {'colsample_bytree': 0.7, 'subsample': 0.8} cv score 0.142 round 836
params: {'colsample_bytree': 0.7, 'subsample': 0.9} cv score 0.1431 round 1296
params: {'colsample_bytree': 0.7, 'subsample': 1.0} cv score 0.1444 round 1595
params: {'colsample_bytree': 0.8, 'subsample': 0.7} cv score 0.1448 round 1118
params: {'colsample_bytree': 0.8, 'subsample': 0.8} cv score 0.1468 round 947
params: {'colsample_bytree': 0.8, 'subsample': 0.9} cv score 0.1492 round 1043
params: {'colsample_bytree': 0.8, 'subsample': 1.0} cv score 0.1465 round 1471
params: {'colsample_bytree': 0.9, 'subsample': 0.7} cv score 0.1463 round 750
params: {'colsample_bytree': 0.9, 'subsample': 0.8} cv score 0.1445 round 1079
params: {'colsample_bytree': 0.9, 'subsample': 0.9} cv score 0.1435 round 1096
params: {'colsample_bytree': 0.9, 'subsample': 1.0} cv score 0.1501 round 1994
params: {'colsample_bytree': 1.0, 'subsample': 0.7} cv s

# grid search for regularization (gamma and lambda (L2))

In [10]:
gamma            = [gamma/100     for gamma in range(0, 50, 5)]
reg_lambda       = [reg_lambda/10    for reg_lambda in range(10,50,5)]
params           = helper_cv(
                        params,
                        dtrain,
                        num_boost_round,
                        seed   = 0,
                        nfold  = 5,
                        metrics= 'rmse',
                        early_stopping_rounds = 50,
                        gamma = gamma,
                        reg_lambda = reg_lambda)

params: {'gamma': 0.0, 'reg_lambda': 1.0} cv score 0.142 round 836
params: {'gamma': 0.0, 'reg_lambda': 1.5} cv score 0.1428 round 1293
params: {'gamma': 0.0, 'reg_lambda': 2.0} cv score 0.1435 round 997
params: {'gamma': 0.0, 'reg_lambda': 2.5} cv score 0.1431 round 956
params: {'gamma': 0.0, 'reg_lambda': 3.0} cv score 0.1451 round 1284
params: {'gamma': 0.0, 'reg_lambda': 3.5} cv score 0.1422 round 1282
params: {'gamma': 0.0, 'reg_lambda': 4.0} cv score 0.1412 round 1386
params: {'gamma': 0.0, 'reg_lambda': 4.5} cv score 0.1418 round 1584
params: {'gamma': 0.05, 'reg_lambda': 1.0} cv score 0.1558 round 822
params: {'gamma': 0.05, 'reg_lambda': 1.5} cv score 0.1581 round 1806
params: {'gamma': 0.05, 'reg_lambda': 2.0} cv score 0.1576 round 1477
params: {'gamma': 0.05, 'reg_lambda': 2.5} cv score 0.1577 round 746
params: {'gamma': 0.05, 'reg_lambda': 3.0} cv score 0.1601 round 902
params: {'gamma': 0.05, 'reg_lambda': 3.5} cv score 0.1569 round 1515
params: {'gamma': 0.05, 'reg_lambda

# test reducing learning rate and increase num_boost_round

In [11]:
# reduce learning rate and increase num_boost_round
eta                      = [eta/100     for eta in range(1, 11, 1)]
num_boost_round          = 5000
params       = helper_cv(
                        params,
                        dtrain,
                        num_boost_round,
                        seed   = 0,
                        nfold  = 5,
                        metrics= 'rmse',
                        early_stopping_rounds = 50,
                        eta  = eta)
print(params)
num_boost_round       = helper_cv(
                        params,
                        dtrain,
                        num_boost_round,
                        seed   = 0,
                        nfold  = 5,
                        metrics= 'rmse',
                        early_stopping_rounds = 50)

params: {'eta': 0.01} cv score 0.1445 round 4999
params: {'eta': 0.02} cv score 0.1394 round 4988
params: {'eta': 0.03} cv score 0.1392 round 3993
params: {'eta': 0.04} cv score 0.1405 round 2947
params: {'eta': 0.05} cv score 0.1405 round 2078
params: {'eta': 0.06} cv score 0.1418 round 2033
params: {'eta': 0.07} cv score 0.1413 round 2117
params: {'eta': 0.08} cv score 0.1432 round 2211
params: {'eta': 0.09} cv score 0.1416 round 1363
params: {'eta': 0.1} cv score 0.1412 round 1386
Best params: {'eta': 0.03} 0.1391724
{'max_depth': 4, 'min_child_weight': 3, 'eta': 0.03, 'gamma': 0.0, 'reg_lambda': 4.0, 'subsample': 0.8, 'colsample_bytree': 0.7}


# train and predict using optimized hyperparameters

In [12]:
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round
    )

In [13]:
predictions = best_model.predict(dtest)

In [14]:
mean_squared_error(predictions, y_test,squared=False)

0.1320883629186027